## Introduction to Electronic Structure Theory
Electronic Structure Theory describes the motions of electrons in atoms or molecules. Because the electrons are so small, one needs to use quantum mechanics to solve for their motion.
### Schrodinger Equation For Molecules
$$H\Psi=E\Psi$$
$$\Psi = \Psi(\vec{r_1}, \omega_1, \vec{r_2}, \omega_2,...) = \Psi(\vec{x_1}, \vec{x_2},...)$$

* The state of an electron is represented by a wavefunction $\Psi(\vec{x})$ where $\vec{x} = (\vec{r}, \omega)$; $\vec{r} = (x, y, z) = (r, \theta, \phi)$ represents the position of the electron and $\omega$ represents represents the spin of the electron.  
* Note that elementary particles like electron carry an intrinsic form of angular momentum called Spin, which cannot be described by a function of positional variables $\vec{r}$, and is somewhat analogous to earth rotating around its own axis.
* Since electrons (fermions) are indistinguishable, the wavefunction $\Psi(\vec{x_1}, \vec{x_2}, ...)$ must be anti-symmetric to satisfy Pauli's Exclusion Principle.

### Molecular Hamiltonian
$$\hat{H} = \underset{\text{Nuclear K.E.}}{\hat{T}_N(R)} + \underset{\text{Electronic K.E.}}{\hat{T}_e(r)} + \underset{\text{Elec/Nuc attraction}}{\hat{V}_{eN}(r, R)} + \underset{\text{Nuc/Nuc repulsion}}{\hat{V}_{NN}(R)} + \underset{\text{Elec/Elec repulsion}}{\hat{V}_{ee}(r)}$$

### Born-Oppenheimer Approximation
* Since Nuclei are much larger than electrons, consider the nuclei to be fixed classical point charges. 
* Therefore, Nuclear coordinates $R$ become fixed parameters; i.e. $\hat{T}_N(R) \approx 0$, $\hat{V}_{NN}(R)$ is a constant and $\hat{V}_{eN}(r; R)$ depends only parametrically on $R$.
* This results in Electronic Hamiltonian given as

$$\hat{H}_{el} = \underset{\text{One-Electron Operators}}{\underset{\text{Electronic K.E.}}{\hat{T}_e(r)}  + \underset{\text{Elec/Nuc attraction}}{\hat{V}_{eN}(r; R)}} + \underset{\text{Two-Electron Operator}}{\underset{\text{Elec/Elec repulsion}}{\hat{V}_{ee}(r)}} = \underset{\text{One-Electron Operator}}{\sum_{i}\hat{h}(i)} + \underset{\text{Two-Electron Operator}}{\sum_{i\ne j}\hat{v}(i, j)}$$

### Potential Energy Surfaces
* For a given configuration of the Nuclei (fixed $R$), we aim to find the the energy eigenstates $|E_i\rangle$ and the corresponding energy eigenvalues $E_i$ of the hamiltonian $H_e$.
* We can solve this equation for a range of nuclear configurations($R$) to map out the Potential Energy Surface of the molecule.

### Orbitals and Chemical Basis Sets
$$\underset{\text{Spin Orbital}}{\chi(\vec{x})} = \underset{\text{Atomic Orbital}}{\Phi(\vec{r})} \cdot \underset{\text{Spin Function}}{\sigma(\omega)} $$
* A Spin Orbital is a one-electron wavefunction which describes the position and spin of an electron in an atom. The spatial component of this one-electron function is called Atomic Orbital.
* A Basis Set is a set of functions, called basis functions, such that any elecronic wavefunction can be (approximately) represented as a linear combination of anti-symmetrized product of basis functions. 
* Chemical basis sets are usually composed of spin orbital functions, examples of which include Gaussion-type Orbitals (GTO), Slater-type Orbitals (STO), Slater Type Orbital-n Gaussians (STO-nG) etc. 

### Slater Determinant
* A slater determinant is an anti-symmetrized product of one or more spin orbitals. For example, Let $\{\chi_{P}(x)\}$ be a basis of M orthonormal spin orbitals, a normalized slater determinant for N electrons can be written as:

$$
\begin{eqnarray}
	\left|\chi_{P_1},\chi_{P_2},...,\chi_{P_N}\right| &=& \frac{1}{\sqrt{N!}} 
		\left|
			\begin{array}{cccc}
				\chi_{P_1}(\vec{x_1}) & \chi_{P_2}(\vec{x_1}) & \ldots & \chi_{P_N}(\vec{x_1}) \\
				\chi_{P_1}(\vec{x_2}) & \chi_{P_2}(\vec{x_2}) & \ldots & \chi_{P_N}(\vec{x_2}) \\
				\vdots            & \vdots            & \ddots & \vdots            \\
				\chi_{P_1}(\vec{x_N}) & \chi_{P_2}(\vec{x_N}) & \ldots & \chi_{P_N}(\vec{x_N}) \\
			\end{array}
		\right|
\end{eqnarray}
$$

### Second Quantization / Occupation Number Represenation

* We can also represent the slater determinant by an *occupation-number vector* $|k\rangle$ s.t.

$$ 
|k\rangle =|k_1, k_2, \ldots, k_M\rangle, k_P = 
\begin{cases}
    1, \  \chi_{P}\ \text{occupied i.e. present in slater determinant}\\
    0, \  \chi_{P}\ \text{unoccupied i.e. absent in slater determinant}
\end{cases}
 $$

* This representation is also called Second Quantization representation, where instead of asking the question "*Which electron is in which state?*", we ask the question "*How many particles are there in each state?*".

* The Second Quantization formalism has Creation ($a_p^\dagger$) and Annihilation ($a_p$) operators to add or remove particles from the many-body electronic wavefunction while maintaining the anti-symmetric property. The action of the operators on determinants is given by
$$
\begin{split} 
a_{p}^{\dagger}|\cdots,k_{p-1},\ k_{p},\ k_{p+1},\cdots\rangle &= (-1)^{\sum_{i < p}k_i}\ (1-&k_p) &|\cdots,k_{p-1},\ 1-k_p,\ k_{p+1},\cdots\rangle \\   
a_{p}|\cdots,k_{p-1},\ k_{p},\ k_{p+1},\cdots\rangle &= (-1)^{\sum _{i < p}k_i}& k_p &|\cdots,k_{p-1},\ 1-k_{p},\ k_{p+1},\cdots\rangle
\end{split}
$$

### Molecular Hamiltonian in Second Quantization
* *One-/Two- Electron Operators* are of the form $\sum_{i}\hat{h}(i)$ / $\sum_{i\ne j}\hat{v}(i, j)$ such that every individual term can only affect one/two electrons by changing their orbitals with some probability. 
 
* Therefore, Given a basis set $\{\chi_{M}(x)\}$, the Electronic Molecular Hamiltonian in Second Quantization is given as 
$$
\hat{H}_{el} = \sum_{PQ}h_{PQ}a_{P}^\dagger a_{Q} + \frac{1}{2}\sum_{PQRS}g_{PQRS}a_{P}^\dagger a_{R}^\dagger a_{S} a_{Q}
$$
* Applied to an electronic state, the Hamiltonian produces a linear combination of the original state with states generated by single and double electron excitations from this state. With each such excitation, there is an associated amplitude $h_{PQ}$ or $g_{PQRS}$, which represents the probability
of this event happening. These probability amplitudes are calculated from the spin orbitals $\{\chi_{M}(x)\}$ and the one- and two-electron operators $\sum_{i}\hat{h}(i)$ and $\sum_{i\ne j}\hat{v}(i, j)$.
* The coefficients $h_{PQ}$ and $g_{PQRS}$ are precomputed for a number of molecules and common chemical basis sets and can be easily obtained using quantum computational chemistry packages like OpenFermion (see below).

## Introduction To OpenFermion
OpenFermion is a quantum computational chemistry package that allows for the easy representation and manipulation of operators on strongly-correlated fermionic, bosonic, and qubit systems. Beginning with an interface to common
electronic structure packages, it simplifies the translation between a molecular specification and a quantum circuit for solving or studying the electronic structure problem on a quantum computer, minimizing the amount of domain expertise required to enter the field.

In [1]:
#Install Openfermion
!pip3 install openfermion --quiet

## Load Molecular Data For $H_2$
### The `MolecularData` Class
OpenFermion uses the `MolecularData` class to store data from computational  chemistry packages. This class has additionally a load and save function, as this data often takes a significant amount of time to prepare. To load stored data, we need to state the geometry of the molecule (in cartesian co-ordinates), the basis in which the molecule is stored, and its charge and multiplicity (i.e. $2n+1$, where $n$ is the number of unpaired spins). OpenFermion also requests a description of the molecule studied

Let's load the molecular data for H2, which is already provided with the package.

In [2]:
import openfermion
diatomic_bond_length = .7414

geometry = [('H', (0., 0., 0.)), 
            ('H', (0., 0., diatomic_bond_length))]

basis = 'sto-3g'
multiplicity = 1
charge = 0
description = format(diatomic_bond_length)

molecule = openfermion.MolecularData(
    geometry,
    basis,
    multiplicity,
    description=description)
molecule.load()

print(molecule.name)

H2_sto-3g_singlet_0.7414


In addition to the above data, and being able to generate the Hamiltonian of the target molecule, the MolecularData class contains various information about classical computational approximations of the ground state energy.



In [3]:
print("Bond Length in Angstroms: {}".format(diatomic_bond_length))
print("Hartree Fock (mean-field) energy in Hartrees: {}".format(molecule.hf_energy))
print("FCI (Exact) energy in Hartrees: {}".format(molecule.fci_energy))

Bond Length in Angstroms: 0.7414
Hartree Fock (mean-field) energy in Hartrees: -1.116684386906734
FCI (Exact) energy in Hartrees: -1.137270174625328


## Get Molecular Hamiltonian For $H_2$
### The `InteractionOperator` Class
The `InteractionOperator` class provides funcionality to efficiently store and manipulate the molecular hamiltonian as 2- and 4- index tensors corresponding to one- and two- electron interaction terms in the hamiltonian. 

Interaction Operators can be obtained by calling `MolecularData.get_molecular_hamiltonian()`


In [4]:
hamiltonian = molecule.get_molecular_hamiltonian()
print(hamiltonian)

() 0.713753990544915
((0, 1), (0, 0)) -1.2524635715927757
((1, 1), (1, 0)) -1.2524635715927757
((2, 1), (2, 0)) -0.4759487172683097
((3, 1), (3, 0)) -0.4759487172683097
((0, 1), (0, 1), (0, 0), (0, 0)) 0.3372443828669511
((0, 1), (0, 1), (2, 0), (2, 0)) 0.09064440419713085
((0, 1), (1, 1), (1, 0), (0, 0)) 0.3372443828669511
((0, 1), (1, 1), (3, 0), (2, 0)) 0.09064440419713085
((0, 1), (2, 1), (0, 0), (2, 0)) 0.09064440419713081
((0, 1), (2, 1), (2, 0), (0, 0)) 0.33173404792821903
((0, 1), (3, 1), (1, 0), (2, 0)) 0.09064440419713081
((0, 1), (3, 1), (3, 0), (0, 0)) 0.33173404792821903
((1, 1), (0, 1), (0, 0), (1, 0)) 0.3372443828669511
((1, 1), (0, 1), (2, 0), (3, 0)) 0.09064440419713085
((1, 1), (1, 1), (1, 0), (1, 0)) 0.3372443828669511
((1, 1), (1, 1), (3, 0), (3, 0)) 0.09064440419713085
((1, 1), (2, 1), (0, 0), (3, 0)) 0.09064440419713081
((1, 1), (2, 1), (2, 0), (1, 0)) 0.33173404792821903
((1, 1), (3, 1), (1, 0), (3, 0)) 0.09064440419713081
((1, 1), (3, 1), (3, 0), (1, 0)) 0.33173

## Convert Molecular Hamiltonian to 4-Qubit Hamiltonian
### The `FermionOperator` Class
The `FermionOperator` class is used to represent the operators on fermionic systems as a sum of creation and annihilation operators. In order to support fast addition of FermionOperator instances, the class is implemented as hash table (python dictionary). The keys of the dictionary encode the strings of ladder operators and values of the dictionary store the coefficients. Each ladder operator is represented by a 2-tuple. The first element of the 2-tuple is an int indicating the tensor factor on which the ladder operator acts. The second element of the 2-tuple is Bool: 1 represents creation and 0 represents annihilation. 

### The `QubitOperator` Class
The `QubitOperator` class is used to store linear combinations of Pauli operators on $N$ qubits. Internally, the data about the operator is stored as a dictionary, with the names of individual Pauli operators used as keys.

### Transformations Between `FermionOperator` and `QubitOperator`
OpenFermion provides utiliites for common transformations like *Jordan-Wigner* and *Bravyi-Kitaev* in order to convert a `FermionOperator` to `QubitOperator`

In [5]:
qubit_hamiltonian = openfermion.jordan_wigner(openfermion.get_fermion_operator(hamiltonian))
print(qubit_hamiltonian)

(-0.09886397351781583+0j) [] +
(-0.04532220209856541+0j) [X0 X1 Y2 Y3] +
(0.04532220209856541+0j) [X0 Y1 Y2 X3] +
(0.04532220209856541+0j) [Y0 X1 X2 Y3] +
(-0.04532220209856541+0j) [Y0 Y1 X2 X3] +
(0.17119774853325848+0j) [Z0] +
(0.16862219143347554+0j) [Z0 Z1] +
(0.12054482186554413+0j) [Z0 Z2] +
(0.16586702396410954+0j) [Z0 Z3] +
(0.1711977485332586+0j) [Z1] +
(0.16586702396410954+0j) [Z1 Z2] +
(0.12054482186554413+0j) [Z1 Z3] +
(-0.22278592890107018+0j) [Z2] +
(0.17434844170557132+0j) [Z2 Z3] +
(-0.22278592890107013+0j) [Z3]
